In [ ]:
# 可刪掉的block
"/Users/cfh00892302/Desktop/myWorkspace/Chinese-Verdict-NLP/dev/label_output_new/merged_data.json"

1. 把資料合併 #暫時不用做這步驟

In [2]:
from information_extraction.utils.base_utils import merge_json
all_json_file_locate_folder = "./dev/label_output_raw"
merged_json_output_folder = "./dev/label_output_new"
merge_json(all_json_file_locate_folder, merged_json_output_folder)

[2023-05-26 14:19:18,522] [    INFO] - Find the following json file: ['個理_文本分類_119.json', '商險_文本分類_25.json'].
[2023-05-26 14:19:18,523] [    INFO] - Merging the file 個理_文本分類_119.json...
[2023-05-26 14:19:18,536] [    INFO] - Length of 個理_文本分類_119.json is 119.
[2023-05-26 14:19:18,538] [    INFO] - Merging the file 商險_文本分類_25.json...
[2023-05-26 14:19:18,543] [    INFO] - Length of 商險_文本分類_25.json is 25.
[2023-05-26 14:19:18,561] [    INFO] - Successful merge all file (len=144) to ./dev/label_output_new/merged_data.json.


In [1]:
file_name = "./dev/label_output_new/final_1000.json"

2. 檢查資料正確性

In [3]:
from information_extraction.utils.base_utils import is_repeat_content_exist
result = is_repeat_content_exist(file_name)
if not result['is_repeat']:
    print("OK! No repeat.")
else:
    print("Duplicate cases happened!!!")
    

[2023-06-16 03:08:00,859] [    INFO] - All data: 1000. Unique data: 1000. Repeat data: 0.


OK! No repeat.


3. 資料前處理 去特殊字元

In [ ]:
from information_extraction.utils.base_utils import regularize_json_file
regularize_json_file(json_file="./dev/label_output_new/final_1000.json", output_path="./information_extraction/data/final_data/")

3. Split into train/eval/test

In [11]:
!python ./information_extraction/utils/split_labelstudio.py \
    --labelstudio_file ./information_extraction/data/final_data/regularized_data.json \
    --save_dir ./information_extraction/data/final_data/ \
    --split_ratio 0.8 0.1 0.1 

/home/ubuntu/work/venv_pytorch/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
[2023-06-16 03:12:35,884] [   DEBUG] - This dir = /home/ubuntu/work/Chinese-Verdict-NLP
[2023-06-16 03:12:35,885] [    INFO] - Converting regularized_data.json into ./information_extraction/data/final_data/...
[2023-06-16 03:12:36,027] [   DEBUG] - in do_split, len(dataset)=1000
[2023-06-16 03:12:36,027] [   DEBUG] - In convert_format, data len = 800
[2023-06-16 03:12:36,033] [   DEBUG] - In convert_format, data len = 100
[2023-06-16 03:12:36,034] [   DEBUG] - In convert_format, data len = 100
[2023-06-16 03:12:36,035] [   DEBUG] - len(training_data.txt) = 2400
[2023-06-16 03:12:36,343] [   DEBUG] - len(eval_data.txt) = 300
[2023-06-16 03:12:36,378] [   DEBUG] - len(testing_data.txt) = 300


4. training

In [ ]:
# gpu test on full data
!python3 ./information_extraction/model/finetune.py  \
    --device gpu \
    --logging_steps 10 \
    --save_steps 100 \
    --eval_steps 100 \
    --seed 42 \
    --model_name_or_path uie-base  \
    --train_path ./information_extraction/data/final_data/training_data.txt \
    --dev_path ./information_extraction/data/final_data/eval_data.txt  \
    --max_seq_len 512  \
    --read_data_method chunk \
    --per_device_eval_batch_size 16 \
    --per_device_train_batch_size  16 \
    --multilingual True \
    --num_train_epochs 5 \
    --learning_rate 1e-5 \
    --label_names 'start_positions' 'end_positions' \
    --do_train \
    --do_eval \
    --do_export \
    --overwrite_output_dir \
    --disable_tqdm True \
    --metric_for_best_model eval_f1 \
    --load_best_model_at_end  True \
    --save_total_limit 1